In [116]:
import numpy as np
import pandas as pd
import re
import requests
import time
from collections import defaultdict
from datetime import datetime
from bs4 import BeautifulSoup

In [117]:
df_race_results = pd.DataFrame(columns = [
    'RACE',
    'YEAR',
    'STAGE_ID',
    'STAGE_DATE',
    'STAGE_NAME',
    'STAGE_PROFILE',
    'STAGE_TYPE',
    'RANK',
    'RIDER',
    'TEAM',
    'UCI_POINTS',
    'PCS_POINTS',
    'UPDATE_TIMESTAMP'
], index = [0])

In [118]:
def get_startlist(link):
    
    URL = "https://www.procyclingstats.com/race/volta-ao-algarve/2025/startlist/alphabetical"
    # link
    
    page = requests.get(URL)
    soup = BeautifulSoup(page.content, "html.parser")

    title = soup.title.text
    if title.startswith("Page not found"):
        return []

    startlist_table = soup.find_all("table", class_ = "basic")[0]
    rows = startlist_table.find("tbody").find_all("tr")

    startlist = []

    for i, row in enumerate(rows):
        cols = row.find_all("td")
        startlist.append(cols[1].text.strip())

    return startlist

In [119]:
RIDERS = pd.read_csv("../data/riders.csv")
RIDER_TEAMS = pd.read_csv("../data/rider_teams.csv")
MANAGERS = pd.read_csv("../data/managers.csv")
MANAGER_TEAMS = pd.read_csv("../data/manager_teams.csv")
MANAGER_CHEAPO_TEAMS = pd.read_csv("../data/manager_cheapo_teams.csv")
POINTS_SYSTEM = pd.read_csv("../data/points_system.csv")
RACES = pd.read_csv("../data/races.csv")
RESULTS_2023 = pd.read_csv("../data/results/results_2023_full.csv")
RESULTS_2024 = pd.read_csv("../data/results/results_2024_full.csv")
RESULTS_2025 = pd.read_csv("../data/results/results_2025_full.csv")
CHEAPO_BANS = pd.read_csv("../data/cheapo_bans.csv")
startlist = get_startlist("a")

In [120]:
TEAMS_OVERVIEW = pd.merge(MANAGER_TEAMS, RIDERS, how = "left", left_on = "RiderName", right_on = "RiderName_Zweeler")
TEAMS_OVERVIEW = TEAMS_OVERVIEW[["ManagerName", "RiderName_PCS", "RiderPrice"]]

def pad_price(price, name):
    padded_price = ("000000" + str(price))[::-1][0:5][::-1]
    name_added_price = padded_price + name
    return name_added_price

TEAMS_OVERVIEW["RankCol"] = TEAMS_OVERVIEW.apply(lambda row: pad_price(row["RiderPrice"], row["RiderName_PCS"]), axis = 1)
TEAMS_OVERVIEW["TeamRank"] = (TEAMS_OVERVIEW.groupby("ManagerName")["RankCol"].rank(method = "dense", ascending = False).astype(int))

nums = list(TEAMS_OVERVIEW["TeamRank"].unique())
nums.sort()
OVERVIEW = pd.DataFrame(nums, columns = ["Rank"])

def get_rider(manager, rank):
    R = TEAMS_OVERVIEW.loc[(TEAMS_OVERVIEW["ManagerName"] == manager) & (TEAMS_OVERVIEW["TeamRank"] == rank)]
    return list(R["RiderName_PCS"].unique())[0]

ms = list(TEAMS_OVERVIEW["ManagerName"].unique())

for m in ms:
    OVERVIEW[m] = OVERVIEW.apply(lambda row: get_rider(m, row["Rank"]), axis = 1)


txt = OVERVIEW.to_html()

In [121]:
for rider in startlist:
    txt.replace("<td>" + rider + "</td>", "<td class=""startlist>" + rider + "</td")

with open("../Løbsstartlister/Volta.md", "w") as file:
    file.write(txt)